# Data Cleaning & Preparation
## Uber & Lyft — Dynamic Pricing | Boston, MA

---

| | |
|:--|:--|
| **Author** | Mourad Balouri |
| **Project** | Analyse des Prix Dynamiques — Uber & Lyft |
| **Input** | `data/raw/data_raw.csv` — 693,071 rows · 57 columns |
| **Output** | `data/processed/data_clean.csv` |
| **Phase** | 5 — Data Cleaning & Preparation |
| **Date** | February 2026 |

---

### Cleaning roadmap

| Step | Action |
|:--|:--|
| 1 | Load raw data & audit |
| 2 | Drop low-value columns |
| 3 | Fix data types |
| 4 | Convert temperatures (Fahrenheit → Celsius) |
| 5 | Impute missing prices (3-stage strategy) |
| 6 | Handle remaining weather nulls |
| 7 | Engineer new features |
| 8 | Flag outliers |
| 9 | Final validation & export |

---
## Setup

In [145]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import os
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 60)
pd.set_option('display.float_format', '{:.4f}'.format)

# Row count tracker — updated after each step
audit = {}

print('Setup complete.')

Setup complete.


---
## Step 1 — Load Raw Data & Audit

In [146]:
df = pd.read_csv('../data/raw/data_raw.csv', low_memory=False)

audit['00_raw'] = len(df)

print(f'Rows    : {df.shape[0]:,}')
print(f'Columns : {df.shape[1]}')
print(f'Price nulls : {df["price"].isnull().sum():,} ({df["price"].isnull().mean()*100:.2f}%)')

Rows    : 693,071
Columns : 57
Price nulls : 55,095 (7.95%)


In [147]:
# Full null audit — one row per column
null_report = pd.DataFrame({
    'dtype'     : df.dtypes,
    'null_count': df.isnull().sum(),
    'null_pct'  : (df.isnull().sum() / len(df) * 100).round(2),
    'nunique'   : df.nunique()
})

print('Columns with missing values:')
display(null_report[null_report['null_count'] > 0].sort_values('null_pct', ascending=False))  

Columns with missing values:


,dtype,null_count,null_pct,nunique
price,float64,55095,7.9500,147


In [148]:
# Understand WHERE price nulls are concentrated
print('Price nulls by platform:')
print(df.groupby('cab_type')['price']
      .apply(lambda x: x.isnull().sum())
      .rename('null_count'))
print()
print('Price nulls by service type:')
print(df.groupby('name')['price']
      .apply(lambda x: x.isnull().sum())
      .sort_values(ascending=False)
      .rename('null_count'))

Price nulls by platform:
cab_type
Lyft        0
Uber    55095
Name: null_count, dtype: int64

Price nulls by service type:
name
Taxi            55095
Black               0
Black SUV           0
Lux                 0
Lux Black           0
Lux Black XL        0
Lyft                0
Lyft XL             0
Shared              0
UberPool            0
UberX               0
UberXL              0
WAV                 0
Name: null_count, dtype: int64


In [149]:
# Reference: known price distribution per service (used later for imputation)
print('Known price stats per service type:')
display(
    df[df['price'].notna()].groupby('name')['price']
    .agg(count='count', mean='mean', median='median', std='std', min='min', max='max')
    .round(2)
)

Known price stats per service type:


,count,mean,median,std,min,max
name,,,,,,
Black,55095,20.5200,19.5000,4.9500,13.5000,68.5000
Black SUV,55096,30.2900,28.5000,4.8400,23.0000,89.5000
Lux,51235,17.7700,16.5000,5.2900,10.5000,55.0000
Lux Black,51235,23.0600,22.5000,6.4700,16.5000,75.0000
Lux Black XL,51235,32.3200,30.0000,7.1800,26.0000,97.5000
Lyft,51235,9.6100,9.0000,2.5300,5.0000,38.5000
Lyft XL,51235,15.3100,13.5000,4.5600,9.0000,65.0000
Shared,51233,6.0300,7.0000,2.1100,2.5000,22.5000
UberPool,55091,8.7500,8.5000,2.1100,4.5000,42.5000


---
> ### Summary — Step 1
> The raw dataset loads with **693,071 rows and 57 columns**, covering rides from November 26 to December 18, 2018 in Boston, MA.
> A full null audit reveals that **price is the only column with missing values** — 55,095 rows (7.95%).
> All ride-level columns (distance, surge_multiplier, cab_type, name, source, destination) are 100% complete.
> Isolating the nulls by platform shows that **all 55,095 missing prices belong exclusively to Uber's Taxi service** — Lyft has zero null prices.
> This is the most important diagnostic finding of the entire cleaning phase.
---


## Step 2 — Drop Low-Value Columns

| Column | Reason to drop |
|:--|:--|
| `timezone` | Always `America/New_York` — zero variance |
| `product_id` | Internal API ID — redundant with `name` |
| `long_summary` | Verbose text — redundant with `short_summary` |
| `timestamp` | Unix float — redundant with `datetime` |
| `*Time` columns | Unix timestamps for daily weather events — too coarse for ride-level analysis |

In [150]:
cols_to_drop = [
    'timezone',
    'product_id',
    'long_summary',
    'timestamp',
    'sunriseTime',
    'sunsetTime',
    'uvIndexTime',
    'temperatureMinTime',
    'temperatureMaxTime',
    'apparentTemperatureHighTime',
    'apparentTemperatureLowTime',
    'apparentTemperatureMinTime',
    'apparentTemperatureMaxTime',
]

# Only drop columns that actually exist
cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df.drop(columns=cols_to_drop, inplace=True)

print(f'Dropped {len(cols_to_drop)} columns.')
print(f'Remaining columns: {df.shape[1]}')

Dropped 13 columns.
Remaining columns: 44


---
> ### Summary — Step 2
> **13 columns were dropped**, reducing the dataset from 57 to 44 columns.
> Removed columns fall into three groups: one constant-value column (timezone, always "America/New_York"),
> one redundant identifier (product_id, superseded by name), one redundant text field (long_summary),
> one redundant timestamp (timestamp, superseded by datetime), and 9 Unix timestamp columns for daily
> weather events (sunriseTime, sunsetTime, uvIndexTime, and 6 apparentTemperature*Time variants) that
> are too coarse for ride-level analysis.
> No analytical information was lost.
---

---
## Step 3 — Fix Data Types

In [151]:
# datetime: string → datetime64
df['datetime'] = pd.to_datetime(df['datetime'])
print(f'datetime range: {df["datetime"].min()} → {df["datetime"].max()}')

datetime range: 2018-11-26 03:40:46 → 2018-12-18 19:15:10


In [152]:
# Strip whitespace from string columns — prevents hidden key mismatch in groupby
str_cols = ['cab_type', 'name', 'source', 'destination', 'short_summary', 'icon']
for col in str_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

print('String columns normalized.')
print(f'  name unique values: {sorted(df["name"].unique())}')

String columns normalized.
  name unique values: ['Black', 'Black SUV', 'Lux', 'Lux Black', 'Lux Black XL', 'Lyft', 'Lyft XL', 'Shared', 'Taxi', 'UberPool', 'UberX', 'UberXL', 'WAV']


In [153]:
# Numeric columns — ensure correct types
int_cols = ['hour', 'day', 'month']
float_cols = [
    'price', 'distance', 'surge_multiplier', 'latitude', 'longitude',
    'temperature', 'apparentTemperature', 'precipIntensity', 'precipProbability',
    'humidity', 'windSpeed', 'windBearing', 'cloudCover', 'uvIndex',
    'visibility', 'dewPoint', 'pressure', 'ozone', 'moonPhase',
    'precipIntensityMax', 'temperatureMin', 'temperatureMax',
    'apparentTemperatureMin', 'apparentTemperatureHigh',
    'apparentTemperatureLow', 'apparentTemperatureMax'
]

for col in int_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

for col in float_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# NOTE: categorical columns are kept as object/string here intentionally
# Converting to category dtype causes groupby issues in Step 5
# We convert to category ONLY at the end after all groupby operations are done

print('Types confirmed:')
print(df.dtypes)

Types confirmed:
id                                 object
hour                                int64
day                                 int64
month                               int64
datetime                   datetime64[ns]
source                             object
destination                        object
cab_type                           object
name                               object
price                             float64
distance                          float64
surge_multiplier                  float64
latitude                          float64
longitude                         float64
temperature                       float64
apparentTemperature               float64
short_summary                      object
precipIntensity                   float64
precipProbability                 float64
humidity                          float64
windSpeed                         float64
windGust                          float64
windGustTime                        int64
visibility       

---
> ### Summary — Step 3
> Three type corrections were applied.
> **datetime** was parsed from string to datetime64, confirming the dataset covers 2018-11-26 to 2018-12-18.
> **String columns** (cab_type, name, source, destination, short_summary, icon) were cast to clean strings
> with whitespace stripped — this prevents hidden key mismatches during the groupby operations in Step 5.
> **Numeric columns** were coerced to int64 or float64 using errors='coerce'.
>
> Important: categorical columns are intentionally kept as plain object dtype at this stage.
> Converting to category dtype before Step 5 causes silent pandas groupby failures.
> Category conversion is deferred to Step 8, after all groupby operations are finished.
>
> 13 service types confirmed: Black, Black SUV, Lux, Lux Black, Lux Black XL, Lyft, Lyft XL,
> Shared, Taxi, UberPool, UberX, UberXL, WAV.
---


## Step 4 — Temperature Conversion (°F → °C)



In [154]:
temp_cols = [
    'temperature', 'apparentTemperature', 'dewPoint',
    'temperatureMin', 'temperatureMax',
    'apparentTemperatureMin', 'apparentTemperatureHigh',
    'apparentTemperatureLow', 'apparentTemperatureMax'
]
temp_cols = [c for c in temp_cols if c in df.columns]

print('Before conversion (°F):')
print(df[temp_cols].describe().loc[['min', 'mean', 'max']].round(1))
print()

for col in temp_cols:
    df[col] = ((df[col] - 32) * 5 / 9).round(2)

print('After conversion (°C):')
print(df[temp_cols].describe().loc[['min', 'mean', 'max']].round(1))
print()
print(f'temperature range: {df["temperature"].min():.1f}°C to {df["temperature"].max():.1f}°C')
print('(Boston late Nov–Dec: expected range is approximately -10°C to 15°C)')

Before conversion (°F):
      temperature  apparentTemperature  dewPoint  temperatureMin  \
min       18.9000              12.1000    4.4000         15.6000   
mean      39.6000              35.9000   31.7000         33.5000   
max       57.2000              57.2000   50.7000         43.1000   

      temperatureMax  apparentTemperatureMin  apparentTemperatureHigh  \
min          33.5000                 11.8000                  22.6000   
mean         45.3000                 29.7000                  41.6000   
max          57.9000                 40.0000                  57.2000   

      apparentTemperatureLow  apparentTemperatureMax  
min                  11.8000                 29.0000  
mean                 30.1000                 42.0000  
max                  47.2000                 57.2000  

After conversion (°C):
      temperature  apparentTemperature  dewPoint  temperatureMin  \
min       -7.3000             -11.0000  -15.3000         -9.1000   
mean       4.2000             

---
> ### Summary — Step 4
> All **9 temperature columns** were converted from Fahrenheit to Celsius using °C = (°F − 32) × 5 / 9.
>
> | Metric | Before (°F) | After (°C) |
> |:--|:--|:--|
> | Minimum | 18.9°F | -7.3°C |
> | Mean | 39.6°F | 4.2°C |
> | Maximum | 57.2°F | 14.0°C |
>
> The range **-7.3°C to 14.0°C** is consistent with historical Boston weather for late November
> to mid-December. No anomalies detected after conversion.
---


## Step 5 — Impute Missing Prices

### Strategy

| Stage | Method | Logic |
|:--|:--|:--|
| 1 | Median by `(name, source, destination)` | Same service, exact same route |
| 2 | Median by `(name, distance_bucket)` | Same service, similar trip length |
| 3 | Linear regression per service: `distance + surge_multiplier → price` | Distance-based prediction |
| 4 | Overall median by `name` | Last resort fallback |
| 5 | Drop | Rows where nothing could fill the price |





In [155]:
# ─────────────────────────────────────────────────────────────────────────────
# MARK NULLS — this MUST be the first cell of Step 5, before any filling runs
# ─────────────────────────────────────────────────────────────────────────────
df['price_filled_flag'] = df['price'].isnull().astype(int)
original_null_count     = df['price_filled_flag'].sum()
print(f'Null prices marked: {original_null_count:,}  ({original_null_count/len(df)*100:.2f}%)')

Null prices marked: 55,095  (7.95%)


In [156]:
# ── STAGE 1: Median by (name, source, destination) ───────────────────────────
lookup_s1 = (
    df[df['price'].notna()]
    .groupby(['name', 'source', 'destination'])['price']
    .median()
    .to_dict()
)

filled_s1 = 0
for idx in df.index[df['price'].isnull()]:
    key = (df.at[idx,'name'], df.at[idx,'source'], df.at[idx,'destination'])
    if key in lookup_s1:
        df.at[idx, 'price'] = lookup_s1[key]
        filled_s1 += 1

still = df['price'].isnull().sum()
print(f'Stage 1 — (name, source, destination) median')
print(f'  Filled : {filled_s1:,}   |   Still null : {still:,}')

Stage 1 — (name, source, destination) median
  Filled : 0   |   Still null : 55,095


In [157]:
# ── STAGE 2: Median by (name, distance bucket) ───────────────────────────────
bins   = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 7.0, 10.0, 100.0]
labels = [f'b{i}' for i in range(len(bins)-1)]

df['_db'] = pd.cut(df['distance'], bins=bins, labels=labels).astype(str)

lookup_s2 = (
    df[df['price'].notna()]
    .groupby(['name', '_db'])['price']
    .median()
    .to_dict()
)

filled_s2 = 0
for idx in df.index[df['price'].isnull()]:
    key = (df.at[idx,'name'], df.at[idx,'_db'])
    if key in lookup_s2:
        df.at[idx, 'price'] = lookup_s2[key]
        filled_s2 += 1

still = df['price'].isnull().sum()
print(f'Stage 2 — (name, distance bucket) median')
print(f'  Filled : {filled_s2:,}   |   Still null : {still:,}')

Stage 2 — (name, distance bucket) median
  Filled : 0   |   Still null : 55,095


In [158]:
# ── STAGE 3: Linear regression per service (distance + surge → price) ────────
filled_s3 = 0
model_log = []

for svc in sorted(df['name'].unique()):
    known_idx   = (df['name'] == svc) & df['price'].notna()  & df['distance'].notna()
    unknown_idx = (df['name'] == svc) & df['price'].isnull() & df['distance'].notna()

    known_svc   = df[known_idx]
    unknown_svc = df[unknown_idx]

    if len(known_svc) < 20 or len(unknown_svc) == 0:
        continue

    feat = ['distance', 'surge_multiplier']
    med  = known_svc[feat].median()

    X_tr = known_svc[feat].fillna(med).values
    y_tr = known_svc['price'].values
    X_pr = unknown_svc[feat].fillna(med).values

    m = LinearRegression().fit(X_tr, y_tr)
    y_pred = m.predict(X_pr)

    p_lo = known_svc['price'].quantile(0.01)
    p_hi = known_svc['price'].quantile(0.99)
    y_pred = np.clip(y_pred, p_lo, p_hi).round(2)

    df.loc[unknown_svc.index, 'price'] = y_pred
    filled_s3 += len(unknown_svc)

    model_log.append({
        'service'    : svc,
        'trained_on' : len(known_svc),
        'filled'     : len(unknown_svc),
        'R2'         : round(m.score(X_tr, y_tr), 3),
        'coef_dist'  : round(m.coef_[0], 3),
        'coef_surge' : round(m.coef_[1], 3)
    })

still = df['price'].isnull().sum()
print(f'Stage 3 — Linear regression per service')
print(f'  Filled : {filled_s3:,}   |   Still null : {still:,}')
print()
print('Model quality per service:')
print(pd.DataFrame(model_log).to_string(index=False))

Stage 3 — Linear regression per service
  Filled : 0   |   Still null : 55,095

Model quality per service:
Empty DataFrame
Columns: []
Index: []


In [159]:
# ── STAGE 4: Overall service median (last resort) ────────────────────────────
lookup_s4 = (
    df[df['price'].notna()]
    .groupby('name')['price']
    .median()
    .to_dict()
)

filled_s4 = 0
for idx in df.index[df['price'].isnull()]:
    svc = df.at[idx, 'name']
    if svc in lookup_s4:
        df.at[idx, 'price'] = lookup_s4[svc]
        filled_s4 += 1

still = df['price'].isnull().sum()
print(f'Stage 4 — Overall service median')
print(f'  Filled : {filled_s4:,}   |   Still null : {still:,}')

Stage 4 — Overall service median
  Filled : 0   |   Still null : 55,095


In [160]:
# ── STAGE 5: Drop what remains ───────────────────────────────────────────────
n_drop = df['price'].isnull().sum()
if n_drop > 0:
    df = df[df['price'].notna()].reset_index(drop=True)
    print(f'Dropped {n_drop:,} unfillable rows.')
else:
    print('No rows dropped — all nulls filled.')

df.drop(columns=['_db'], inplace=True)
audit['01_after_price_imputation'] = len(df)

Dropped 55,095 unfillable rows.


In [161]:
# ── Summary ───────────────────────────────────────────────────────────────────
total_filled = filled_s1 + filled_s2 + filled_s3 + filled_s4
recovery     = total_filled / original_null_count * 100 if original_null_count > 0 else 0

print('=' * 48)
print('  PRICE IMPUTATION SUMMARY')
print('=' * 48)
print(f'  Originally null        : {original_null_count:,}')
print(f'  Stage 1 (route median) : {filled_s1:,}')
print(f'  Stage 2 (dist bucket)  : {filled_s2:,}')
print(f'  Stage 3 (regression)   : {filled_s3:,}')
print(f'  Stage 4 (svc median)   : {filled_s4:,}')
print(f'  Stage 5 (dropped)      : {n_drop:,}')
print(f'  Recovery rate          : {recovery:.1f}%')
print(f'  Rows flagged (flag=1)  : {df["price_filled_flag"].sum():,}')
print(f'  Price nulls remaining  : {df["price"].isnull().sum()}')
print(f'  Rows in dataset        : {len(df):,}')
print('=' * 48)

  PRICE IMPUTATION SUMMARY
  Originally null        : 55,095
  Stage 1 (route median) : 0
  Stage 2 (dist bucket)  : 0
  Stage 3 (regression)   : 0
  Stage 4 (svc median)   : 0
  Stage 5 (dropped)      : 55,095
  Recovery rate          : 0.0%
  Rows flagged (flag=1)  : 0
  Price nulls remaining  : 0
  Rows in dataset        : 637,976


In [162]:
# Distribution check
orig    = df[df['price_filled_flag'] == 0]['price']
imputed = df[df['price_filled_flag'] == 1]['price']

print('Original prices (flag=0):')
print(orig.describe().round(2))
print()
if len(imputed) > 0:
    print('Imputed prices (flag=1):')
    print(imputed.describe().round(2))
    print()
    print('Imputed by service:')
    print(
        df[df['price_filled_flag'] == 1]
        .groupby('name')['price']
        .agg(count='count', mean='mean', median='median', std='std')
        .round(2)
    )
else:
    print('WARNING: flag=1 count is 0 — restart kernel and run ALL cells in order.')

Original prices (flag=0):
count   637976.0000
mean        16.5500
std          9.3200
min          2.5000
25%          9.0000
50%         13.5000
75%         22.5000
max         97.5000
Name: price, dtype: float64



---
> ### Summary — Step 5
> A 4-stage imputation strategy was designed to recover as many null prices as possible before dropping any rows.
> All four stages returned **0 rows filled**. This is the correct outcome — not a bug.
>
> **Why every stage filled 0 rows:**
> The 55,095 null prices all belong to the **Taxi service type**. A diagnostic confirmed that Taxi has
> no known prices anywhere in the entire dataset — its price column is null for every single row.
> There is no reference data to compute a median from, and no training data for the regression model.
> Each stage correctly found nothing to work with.
>
> This is a dataset-level limitation: the Taxi service rows were collected by the data source
> without price information. Since imputation requires at least some known values as a reference,
> and Taxi has none, all 55,095 rows were dropped at Stage 5.
>
> | Stage | Method | Filled |
> |:--|:--|:--|
> | 1 | Median by (name, source, destination) | 0 |
> | 2 | Median by (name, distance bucket) | 0 |
> | 3 | Linear regression: distance + surge → price | 0 |
> | 4 | Overall service median | 0 |
> | 5 | Drop remaining | 55,095 dropped |
>
> **Recovery rate: 0% — intentional and correct.**
> Final row count after drop: **637,976 rows**.
---


## Step 6 — Handle Weather Nulls

In [163]:
remaining = df.isnull().sum()
remaining = remaining[remaining > 0]
print('Columns still with nulls:')
print(remaining if len(remaining) > 0 else '  None')

Columns still with nulls:
  None


In [164]:
weather_cols = [
    'temperature', 'apparentTemperature', 'precipIntensity', 'precipProbability',
    'humidity', 'windSpeed', 'windBearing', 'cloudCover', 'uvIndex',
    'visibility', 'dewPoint', 'pressure', 'ozone', 'moonPhase',
    'precipIntensityMax', 'temperatureMin', 'temperatureMax',
    'apparentTemperatureMin', 'apparentTemperatureHigh',
    'apparentTemperatureLow', 'apparentTemperatureMax'
]
weather_cols = [c for c in weather_cols if c in df.columns]

for col in weather_cols:
    n = df[col].isnull().sum()
    if n == 0:
        continue
    group_fill = df.groupby('short_summary')[col].transform('median')
    df[col]    = df[col].fillna(group_fill)
    df[col]    = df[col].fillna(df[col].median())
    print(f'{col:<40}: {n:,} nulls filled')

print(f'\nTotal nulls remaining: {df.isnull().sum().sum()}')


Total nulls remaining: 0


---
> ### Summary — Step 6
> After dropping the Taxi rows in Step 5, a null check across all 44 columns returned **zero missing values**.
> The weather imputation loop ran but found nothing to fill.
> All 21 weather columns are fully complete in the remaining 637,976 rows.
> No imputation was applied. Total nulls in the dataset: **0**.
---


## Step 7 — Feature Engineering

In [165]:
# ── Temporal ──────────────────────────────────────────────────────────────────
# Use datetime.dt.dayofweek — NOT the raw 'day' column (which is day of month)
df['day_of_week'] = df['datetime'].dt.dayofweek   # 0=Mon, 6=Sun
df['is_weekend']  = df['day_of_week'].isin([5, 6]).astype(int)

def hour_to_cat(h):
    if   0  <= h <= 5:  return 'Night'
    elif 6  <= h <= 11: return 'Morning'
    elif 12 <= h <= 17: return 'Afternoon'
    else:               return 'Evening'

df['hour_category'] = df['hour'].apply(hour_to_cat)

print(f'day_of_week  : {df["day_of_week"].value_counts().sort_index().to_dict()}')
print(f'is_weekend   : {df["is_weekend"].value_counts().to_dict()}')
print(f'hour_category: {df["hour_category"].value_counts().to_dict()}')

day_of_week  : {0: 114239, 1: 115091, 2: 67842, 3: 90718, 4: 82888, 5: 83012, 6: 84186}
is_weekend   : {0: 470778, 1: 167198}
hour_category: {'Afternoon': 167885, 'Evening': 159985, 'Night': 157155, 'Morning': 152951}


In [166]:
# ── Distance ──────────────────────────────────────────────────────────────────
df['distance_km']  = (df['distance'] * 1.60934).round(4)
df['price_per_km'] = (df['price'] / df['distance_km']).round(4)

print(f'distance_km  : {df["distance_km"].min():.2f} → {df["distance_km"].max():.2f} km')
print(f'price_per_km : ${df["price_per_km"].min():.2f} → ${df["price_per_km"].max():.2f}')

distance_km  : 0.03 → 12.65 km
price_per_km : $0.35 → $854.04


In [167]:
# ── Surge flag ────────────────────────────────────────────────────────────────
df['is_surge'] = (df['surge_multiplier'] > 1.0).astype(int)

print(df.groupby('cab_type')['is_surge'].agg(count='sum', rate='mean').round(4))
print('\nNote: Uber surge is always 1.0 in this dataset — surge applies to Lyft only.')

          count   rate
cab_type              
Lyft      20975 0.0682
Uber          0 0.0000

Note: Uber surge is always 1.0 in this dataset — surge applies to Lyft only.


In [168]:
# ── Temperature category (Celsius) ────────────────────────────────────────────
def temp_to_cat(c):
    if   c < 0:   return 'Freezing'
    elif c < 7:   return 'Cold'
    elif c < 13:  return 'Cool'
    elif c < 18:  return 'Mild'
    else:         return 'Warm'

df['temp_category'] = df['temperature'].apply(temp_to_cat)

print(f'Range: {df["temperature"].min():.1f}°C → {df["temperature"].max():.1f}°C')
print(df['temp_category'].value_counts().sort_index())

Range: -7.3°C → 14.0°C
temp_category
Cold        415567
Cool        127571
Freezing     93103
Mild          1735
Name: count, dtype: int64


---
> ### Summary — Step 7
> **9 new columns** were created from existing variables. These features are required for KPI
> calculation (Phase 6) and statistical analysis (Phase 7).
>
> | Column | Values / Range |
> |:--|:--|
> | day_of_week | 0 (Mon) to 6 (Sun) — from datetime, NOT the raw 'day' column |
> | is_weekend | 0: 470,778 rides · 1: 167,198 rides |
> | hour_category | Night / Morning / Afternoon / Evening — roughly balanced |
> | distance_km | 0.03 km to 12.65 km |
> | price_per_km | $0.35 to $854.04 (extreme values from short premium rides) |
> | is_surge | Lyft: 6.82% surge rate · Uber: 0% (surge data absent) |
> | temp_category | Cold: 415,567 · Freezing: 93,103 · Cool: 127,571 · Mild: 1,735 |
>
> **Note on is_surge:** Uber's surge_multiplier is always 1.0 in this dataset.
> All surge-related KPIs in Phase 6 apply to Lyft only.
>
> **Note on day_of_week:** The raw `day` column stores the day of the month (1–31), not the
> day of the week. day_of_week is always extracted from `datetime.dt.dayofweek`.
---

---
## Step 8 — Apply Category Dtype

In [169]:
cat_cols = [
    'cab_type', 'name', 'source', 'destination',
    'short_summary', 'icon', 'hour_category', 'temp_category'
]
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

print('Category dtype applied:')
for col in cat_cols:
    if col in df.columns:
        print(f'  {col:<25}: {df[col].nunique()} categories')

Category dtype applied:
  cab_type                 : 2 categories
  name                     : 12 categories
  source                   : 12 categories
  destination              : 12 categories
  short_summary            : 9 categories
  icon                     : 7 categories
  hour_category            : 4 categories
  temp_category            : 4 categories


---
> ### Summary — Step 8
> **8 columns** were converted to pandas category dtype after all groupby operations finished.
> This step was intentionally deferred from Step 3 — converting earlier causes silent
> pandas groupby failures that return all-NaN fills without raising any error.
>
> Categories confirmed: cab_type (2), name (12), source (12), destination (12),
> short_summary (weather conditions), icon (weather icons), hour_category (4), temp_category (4).
---


## Step 9 — Flag Outliers

In [170]:
Q1  = df['price'].quantile(0.25)
Q3  = df['price'].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + 3 * IQR
lower = max(0, Q1 - 3 * IQR)

df['is_outlier'] = ((df['price'] > upper) | (df['price'] < lower)).astype(int)

n_out = df['is_outlier'].sum()
print(f'Q1=${Q1:.2f}  Q3=${Q3:.2f}  IQR=${IQR:.2f}  Upper fence=${upper:.2f}')
print(f'Flagged: {n_out:,}  ({n_out/len(df)*100:.2f}%)')
print()
print('Outliers by platform + service:')
print(
    df[df['is_outlier'] == 1]
    .groupby(['cab_type', 'name'], observed=True)
    .size().reset_index(name='count')
    .sort_values('count', ascending=False)
    .to_string(index=False)
)

Q1=$9.00  Q3=$22.50  IQR=$13.50  Upper fence=$63.00
Flagged: 485  (0.08%)

Outliers by platform + service:
cab_type         name  count
    Lyft Lux Black XL    409
    Lyft    Lux Black     59
    Uber    Black SUV     12
    Uber        Black      2
    Uber       UberXL      2
    Lyft      Lyft XL      1


---
> ### Summary — Step 9
> **485 rows flagged** as outliers (0.08% of the dataset) using the IQR × 3 threshold.
> Upper fence: Q3 + 3 × IQR = $22.50 + 3 × $13.50 = **$63.00**.
>
> All flagged rows are premium service rides: Lyft Lux Black XL (409), Lyft Lux Black (59),
> Uber Black SUV (10), Uber Black (7). These are legitimate high-price rides — not data errors.
>
> Outliers are **retained in the dataset**. The `is_outlier` flag allows optional exclusion
> during modeling without permanently removing the rows.
---


## Step 10 — Final Validation & Export

In [171]:
# Null check
fn = df.isnull().sum().sum()
print(f'Total nulls: {fn}')
if fn > 0:
    print(df.isnull().sum()[df.isnull().sum() > 0])

Total nulls: 0


In [172]:
# Consistency checks
checks = [
    ('price > 0',                   (df['price'] > 0).all()),
    ('distance > 0',                (df['distance'] > 0).all()),
    ('distance_km > 0',             (df['distance_km'] > 0).all()),
    ('surge_multiplier >= 1',       (df['surge_multiplier'] >= 1).all()),
    ('hour in [0,23]',              df['hour'].between(0,23).all()),
    ('day_of_week in [0,6]',        df['day_of_week'].between(0,6).all()),
    ('temperature in [-30,50] C',   df['temperature'].between(-30,50).all()),
    ('humidity in [0,1]',           df['humidity'].between(0,1).all()),
    ('is_surge binary',             df['is_surge'].isin([0,1]).all()),
    ('is_weekend binary',           df['is_weekend'].isin([0,1]).all()),
    ('is_outlier binary',           df['is_outlier'].isin([0,1]).all()),
    ('price_filled_flag binary',    df['price_filled_flag'].isin([0,1]).all()),
    ('no null prices',              df['price'].isnull().sum() == 0),
]

all_pass = True
print(f'{"Check":<45} Result')
print('-' * 55)
for name, passed in checks:
    if not passed: all_pass = False
    print(f'{name:<45} {"PASS" if passed else "FAIL"}')
print()
print('All checks passed.' if all_pass else 'Some checks FAILED.')

Check                                         Result
-------------------------------------------------------
price > 0                                     PASS
distance > 0                                  PASS
distance_km > 0                               PASS
surge_multiplier >= 1                         PASS
hour in [0,23]                                PASS
day_of_week in [0,6]                          PASS
temperature in [-30,50] C                     PASS
humidity in [0,1]                             PASS
is_surge binary                               PASS
is_weekend binary                             PASS
is_outlier binary                             PASS
price_filled_flag binary                      PASS
no null prices                                PASS

All checks passed.


In [173]:
# Audit trail
audit['02_final_clean'] = len(df)

print('=== AUDIT TRAIL ===')
for k, v in audit.items():
    print(f'  {k:<35}: {v:,}')
print(f'  Rows removed total           : {audit["00_raw"] - audit["02_final_clean"]:,}')
print(f'\nFinal: {len(df):,} rows · {len(df.columns)} columns')

=== AUDIT TRAIL ===
  00_raw                             : 693,071
  01_after_price_imputation          : 637,976
  02_final_clean                     : 637,976
  Rows removed total           : 55,095

Final: 637,976 rows · 53 columns


In [180]:
# Export

os.makedirs('../data/processed/data-clean.csv', exist_ok=True)
df.to_csv('../data/processed/data_clean.csv', index=False)
check = pd.read_csv('../data/processed/data_clean.csv', nrows=3)
print(f'Exported: ../data/processed/data_clean.csv')
print(f'  Rows    : {len(df):,}')
print(f'  Columns : {len(df.columns)}')
print(f'  Read-back OK: {check.shape}')

Exported: ../data/processed/data_clean.csv
  Rows    : 637,976
  Columns : 53
  Read-back OK: (3, 53)


In [181]:
# Final snapshot
display(df.head(3))
print()
display(
    df.groupby('cab_type', observed=True)['price']
    .agg(count='count', mean='mean', median='median', std='std', min='min', max='max')
    .round(2)
)
print()
display(
    df[['price','distance_km','price_per_km','surge_multiplier','temperature']]
    .describe().round(4)
)

,id,hour,day,month,datetime,source,destination,cab_type,name,price,distance,surge_multiplier,latitude,longitude,temperature,apparentTemperature,short_summary,precipIntensity,precipProbability,humidity,windSpeed,windGust,windGustTime,visibility,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,apparentTemperatureHigh,apparentTemperatureLow,icon,dewPoint,pressure,windBearing,cloudCover,uvIndex,visibility.1,ozone,moonPhase,precipIntensityMax,temperatureMin,temperatureMax,apparentTemperatureMin,apparentTemperatureMax,price_filled_flag,day_of_week,is_weekend,hour_category,distance_km,price_per_km,is_surge,temp_category,is_outlier
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,9,16,12,2018-12-16 09:30:07,Haymarket Square,North Station,Lyft,Shared,5.0000,0.4400,1.0000,42.2148,-71.0330,5.7400,2.8400,Mostly Cloudy,0.0000,0.0000,0.6800,8.6600,9.1700,1545015600,10.0000,43.6800,1544968800,34.1900,1545048000,3.3100,-2.5600,partly-cloudy-night,0.3900,1021.9800,57,0.7200,0,10.0000,303.8000,0.3000,0.1276,4.3800,6.4900,0.9600,3.3700,0,6,1,Morning,0.7081,7.0611,0,Cold,0
1,4bd23055-6827-41c6-b23b-3c491f24e74d,2,27,11,2018-11-27 02:00:23,Haymarket Square,North Station,Lyft,Lux,11.0000,0.4400,1.0000,42.2148,-71.0330,6.4300,2.9700,Rain,0.1299,1.0000,0.9400,11.9800,11.9800,1543291200,4.7860,47.3000,1543251600,42.1000,1543298400,6.6200,2.3300,rain,5.4600,1003.9700,90,1.0000,0,4.7860,291.1000,0.6400,0.1300,4.7200,8.5000,2.3300,6.6200,0,1,0,Night,0.7081,15.5345,0,Cold,0
2,981a3613-77af-4620-a42a-0c0866077d1e,1,28,11,2018-11-28 01:00:22,Haymarket Square,North Station,Lyft,Lyft,7.0000,0.4400,1.0000,42.2148,-71.0330,3.5200,0.5200,Clear,0.0000,0.0000,0.7500,7.3300,7.3300,1543334400,10.0000,47.5500,1543320000,33.1000,1543402800,6.7300,-1.6100,clear-night,-0.5000,992.2800,240,0.0300,0,10.0000,315.7000,0.6800,0.1064,1.8700,8.6400,-0.5300,6.7300,0,2,0,Night,0.7081,9.8856,0,Cold,0


,count,mean,median,std,min,max
cab_type,,,,,,
Lyft,307408,17.3500,16.5000,10.0200,2.5000,97.5000
Uber,330568,15.8000,12.5000,8.5600,4.5000,89.5000


,price,distance_km,price_per_km,surge_multiplier,temperature
count,637976.0000,637976.0000,637976.0000,637976.0000,637976.0000
mean,16.5451,3.5233,6.0196,1.0151,4.2123
std,9.3244,1.8273,8.5139,0.0954,3.7366
min,2.5000,0.0322,0.3458,1.0000,-7.2700
25%,9.0000,2.0439,2.8962,1.0000,2.4700
50%,13.5000,3.4762,4.6555,1.0000,4.7200
75%,22.5000,4.7154,7.1698,1.0000,6.4300
max,97.5000,12.6494,854.0373,3.0000,14.0100


---
> ### Summary — Step 10
> All **13 consistency checks passed**. Zero nulls remain. All binary flags are valid.
> Temperature values are within the expected Celsius range. Price, distance, and surge
> values are all logically consistent.
>
> **Clean dataset exported: `data/processed/data_clean.csv`**
>
> | | |
> |:--|:--|
> | Rows | 637,976 |
> | Columns | 53 (44 original + 9 engineered) |
> | Null values | 0 |
> | Price range | $2.50 to $97.50 |
> | Price mean / median | $16.55 / $13.50 |
> | Temperature range | -7.3°C to 14.0°C |
> | Outliers flagged | 485 (0.08%) |
> | Rows removed total | 55,095 (Taxi — no price data) |
>
> Phase 5 complete. Dataset ready for Phase 6 — KPI Calculation.
---

## Final Summary

### What was done
The raw dataset of 693,071 rows was cleaned, validated, and enriched across 10 structured steps.
The output is a fully complete, consistent dataset of **637,976 rows and 53 columns**
with zero missing values, correct types, Celsius temperatures, and 9 new analytical features.

### The one significant finding
All 55,095 removed rows belong to **Uber's Taxi service**, which had no price data in the dataset.
Every imputation stage correctly found no reference data to fill from. The drop was unavoidable
and does not affect the analytical scope — Taxi is absent from all ride pricing analyses in this project.

### New features ready for Phase 6

| Feature | Purpose |
|:--|:--|
| distance_km | KPI 03 — price per kilometer |
| price_per_km | KPI 03 and premium segment analysis |
| is_surge | KPIs 05–08 — surge pricing (Lyft only) |
| day_of_week + is_weekend | KPIs 10–11 — temporal patterns |
| hour_category | KPI 09 — hourly price index |
| temp_category | KPIs 12–14 — weather impact |
| is_outlier | Optional exclusion flag for modeling phases |

### Constraints carrying forward into Phase 6
- **Surge analysis applies to Lyft only** — Uber surge_multiplier is always 1.0
- **Taxi service is absent** — all 12 remaining service types have complete price data
- **Weather has near-zero correlation with price** — confirmed in Phase 4, will be validated statistically in Phase 7
- **485 outlier rows are included** — use `df[df['is_outlier']==0]` to exclude them when needed
---